# Working with mlflow and prediction

In [106]:
import os
import sys
sys.path.append(os.path.abspath(os.path.join('../scripts')))
import warnings
warnings.filterwarnings('ignore')
import logging
logging.basicConfig(level=os.environ.get("LOGLEVEL", "INFO"))

In [112]:
import sys
import os
#from ensurepip import version
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error , mean_absolute_error , r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet
from urllib.parse import urlparse
import mlflow
#import mlflow.sklearn
import logging
import warnings

# 
sys.path.append(os.path.abspath(os.path.join('..')))

logging.basicConfig (level = logging .WARN )
logger= logging.getLogger (__name__ )

#Get url from DVC
import dvc.api

path='data/train.csv'
repo ='C:/Users/Ekubay/Documents/Rossmann_Pharmaceutical_sales_Prediction'
rev = 'ver_4'
data_url = dvc.api.get_url(path=path, repo=repo, rev=rev)
#df = pd.read_csv(data_url)

mlflow.set_experiment('prediction')

def eval_metrics ( actual , pred ) :
    rnse = np.sqrt ( mean_squared_error ( actual , pred ) )
    mae = mean_absolute_error ( actual , pred )
    r2 = r2_score ( actual , pred )
    return rnse , mae , r2



2022/09/08 11:57:51 INFO mlflow.tracking.fluent: Experiment with name 'prediction' does not exist. Creating a new experiment.


In [119]:
# main 
if __name__ == "__main__":
    warnings.filterwarnings("ignore")
    np.random.seed(40)
# Read the samrt ad from the remote repository
df_train = pd.read_csv (data_url)
#df_train = get_data('ver_4')
# Log data params
mlflow.log_param ( ' data_url', data_url)
mlflow.log_param ( ' data version ' , rev )
mlflow.log_param ( ' input_rows ' , df_train.shape [0] )
mlflow.log_param ( ' input_cols ' , df_train.shape [ 1 ])

# checking
#print(data['response'])
# Split the data into training and test sets . ( 0.75 , 0.25 ) split .
columns = ['Sales','Store', 'DayOfWeek', 'Open', 'Promo',  'SchoolHoliday', 'Day', 'WeekOfYear','Month', 'Year', 'StoreType',
              'Assortment','CompetitionDistance', 'Promo2']

feature_columns = ['Store', 'DayOfWeek', 'Open', 'Promo',  'SchoolHoliday', 'Day', 'WeekOfYear','Month', 'Year', 'StoreType',
              'Assortment','CompetitionDistance', 'Promo2']

sample_size = 1000
sampled_df = df_train[columns].sample(sample_size)

#
lb = LabelEncoder()
train_x['StoreType'] = lb.fit_transform(train_x['StoreType'])
train_x['Assortment'] = lb.fit_transform(train_x['Assortment'])

train_x.info()
# scaling
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

# split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

# X = x_scaler.fit_transform(train_x)
X = train_x
Y = y_scaler.fit_transform(train_y)
## Separet Test and train files
train_x = sampled_df[feature_columns]
train_y = sampled_df[['Sales']]
train_x.head()

# training
rf_reg = RandomForestRegressor(n_estimators = 100, max_depth=25, random_state=0)
rf_reg.fit(X_train, y_train)

# score of the model
score = rf_reg.score(X_test, y_test)
#print(f"Prediction Score of the Model is {round(score * 100, 2)}%"

#
cols_x = pd.DataFrame (list(train_x.columns))
cols_x.to_csv('features.csv', header = False , index = False )
mlflow.log_artifact('features.csv')

cols_y = pd.DataFrame (list(train_y.columns))
cols_y.to_csv('targets.csv', header = False , index = False )
mlflow.log_artifact('targets.csv')

# alpha = float ( sys.argv [ 1 ] ) if len ( sys.argv ) > 1 else 0.5
# l1_ratio = float ( sys.argv [ 2 ] ) if len ( sys.argv ) > 2 else 0.5
# lr = ElasticNet(alpha = alpha , l1_ratio = l1_ratio , random_state = 42 )
# lr.fit ( train_x , train_y )

#Write scores to a file
# with open("metrics.txt", 'w') as outfile:
#     outfile.write("an_absolute_error: %2.1f%%\n" % rnse)
#     outfile.write("r2_score: %2.1f%%\n" %r2)
        

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1000 entries, 332285 to 708709
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Store                1000 non-null   int64  
 1   DayOfWeek            1000 non-null   int64  
 2   Open                 1000 non-null   int64  
 3   Promo                1000 non-null   int64  
 4   SchoolHoliday        1000 non-null   int64  
 5   Day                  1000 non-null   int64  
 6   WeekOfYear           1000 non-null   int64  
 7   Month                1000 non-null   int64  
 8   Year                 1000 non-null   int64  
 9   StoreType            1000 non-null   int32  
 10  Assortment           1000 non-null   int32  
 11  CompetitionDistance  1000 non-null   float64
 12  Promo2               1000 non-null   int64  
dtypes: float64(1), int32(2), int64(10)
memory usage: 101.6 KB


### Runing the mlflow from notebook

In [ ]:
!mlflow ui